In [1]:
from fforma.metrics import WeightedPinballLoss
from fforma.meta_learner import MetaLearnerNN
from fforma.meta_results_r_data import prepare_fforma_data

from ESRNN.m4_data import prepare_m4_data, seas_dict
from ESRNN.utils_evaluation import evaluate_prediction_owa

from fforma.utils import (
    FactorQuantileRegressionAveraging,
    LassoQuantileRegressionAveraging,
    evaluate_forecasts,
    freqs
)

import pandas as pd
import itertools
import numpy as np
import matplotlib.pyplot as plt
from fforma import FFORMA

In [2]:
X_train_df, preds_train_df, y_train_df, X_test_df, preds_test_df = prepare_fforma_data('data/hyndman_data',
                                                                                        dataset_name='Hourly')
#dataset_name=None for all data

In [3]:
X_train_df2, preds_train_df2, y_train_df2, X_test_df2, preds_test_df2 = prepare_fforma_data('data/hyndman_data',
                                                                                        dataset_name='Daily')
#dataset_name=None for all data

In [4]:
X_train_df = X_train_df.append(X_train_df2).reset_index(drop=True)
preds_train_df = preds_train_df.append(preds_train_df2).reset_index(drop=True)
y_train_df = y_train_df.append(y_train_df2).reset_index(drop=True)
X_test_df = X_test_df.append(X_test_df2).reset_index(drop=True)
preds_test_df = preds_test_df.append(preds_test_df2).reset_index(drop=True)

In [5]:
# def long_to_horizontal(long_df):
#     horizontal_df = pd.DataFrame(columns=long_df.columns)
#     cols_to_parse = list(set(long_df.columns)-set(['unique_id']))
#     long_df = long_df.sort_values(['unique_id','ds']).reset_index(drop=True)
#     unique_ids = long_df['unique_id'].unique()
#     n_series = len(unique_ids)
#     max_len = long_df.groupby('unique_id')['ds'].count().max()
    
#     dict_df = {'unique_id':unique_ids,
#                'ds':list(range(1, max_len+1))}
#     padding_dict = list(itertools.product(*list(dict_df.values())))
#     padding_dict = pd.DataFrame(padding_dict, columns=list(dict_df.keys()))
#     df_padded = padding_dict.merge(long_df, on=['unique_id','ds'], how='outer')
#     df_padded = df_padded.sort_values(['unique_id','ds']).reset_index(drop=True)
    
#     for col in cols_to_parse:
#         values = df_padded[col].values
#         values = values.reshape((n_series, max_len))
#         values = values.tolist()
#         horizontal_df[col] = values

#     horizontal_df['unique_id'] = unique_ids
        
#     return horizontal_df

In [6]:
# ver = long_to_horizontal(preds_train_df)

In [7]:
# def wide_to_long(df, lst_cols, fill_value='', preserve_index=False):
#     # make sure `lst_cols` is list-alike
#     if (lst_cols is not None
#         and len(lst_cols) > 0
#         and not isinstance(lst_cols, (list, tuple, np.ndarray, pd.Series))):
#         lst_cols = [lst_cols]
#     # all columns except `lst_cols`
#     idx_cols = df.columns.difference(lst_cols)
#     # calculate lengths of lists
#     lens = df[lst_cols[0]].str.len()
#     # preserve original index values    
#     idx = np.repeat(df.index.values, lens)
#     # create "exploded" DF
#     res = (pd.DataFrame({
#                 col:np.repeat(df[col].values, lens)
#                 for col in idx_cols},
#                 index=idx)
#              .assign(**{col:np.concatenate(df.loc[lens>0, col].values)
#                             for col in lst_cols}))
#     # append those rows that have empty lists
#     if (lens == 0).any():
#         # at least one list in cells is empty
#         res = (res.append(df.loc[lens==0, idx_cols], sort=False)
#                   .fillna(fill_value))
#     # revert the original index order
#     res = res.sort_index()
#     # reset index if requested
#     if not preserve_index:        
#         res = res.reset_index(drop=True)
#     return res

In [8]:
# vasis_names = ['ds','auto_arima_forec','ets_forec','nnetar_forec',
#          'tbats_forec','stlm_ar_forec','rw_drift_forec',
#          'theta_forec', 'naive_forec', 'snaive_forec']

In [9]:
# explode(ver,vasis_names)

In [10]:
unique_ids = X_train_df['unique_id'].unique()
unique_ids

array(['H1', 'H2', 'H3', ..., 'D4225', 'D4226', 'D4227'], dtype=object)

In [11]:
dataset_name = 'Hourly'
directory = './data/'

_, _, _, y_test_df = prepare_m4_data(dataset_name='Hourly',
                                                      directory=directory,
                                                      num_obs=100_000)
_, _, _, y_test_df2 = prepare_m4_data(dataset_name='Daily',
                                                      directory=directory,
                                                      num_obs=100_000)

In [12]:
y_test_df = y_test_df.append(y_test_df2)
y_test_df = y_test_df[y_test_df['unique_id'].isin(unique_ids)]
y_test_df = y_test_df.reset_index(drop=True)

In [13]:
y_test_df['ds'] = y_test_df[['unique_id','ds']].groupby('unique_id').cumcount()+1 

In [14]:
nn_params = {#'layers': [100, 100],
             'layers': [200, 100, 50, 25, 10],
             'dropout': 0.1,
             'epochs': 2,
             'batch_size': 8,
             'learning_rate': 0.001,
             'gradient_eps': 1e-8,
             'weight_decay': 0,
             'lr_scheduler_step_size': 200,
             'lr_decay': 0.1,
             'use_softmax':True,
             'loss_function': WeightedPinballLoss(0.20)}

In [15]:
model = MetaLearnerNN(nn_params)

In [16]:
model.fit(X_train_df, preds_train_df, y_train_df)

========= Epoch 0 finished =========
Training time: 2.71227
Training loss: 95.42896
========= Epoch 1 finished =========
Training time: 2.6564
Training loss: 88.02976


In [17]:
# ver = model.pad_long_df(preds_test_df)
y_hat = model.predict(X_test_df, preds_test_df)

In [22]:
len(y_hat)

79050

In [18]:
fforma = FFORMA(meta_learner_params=nn_params,
                meta_learner=MetaLearnerNN,
                metric='owa',
                random_seed=1)

In [19]:
fforma.fit(X_train_df, preds_train_df, y_train_df)

========= Epoch 0 finished =========
Training time: 2.76271
Training loss: 95.42896
========= Epoch 1 finished =========
Training time: 2.79917
Training loss: 88.02976


In [21]:
len(y_test_df)

79036

In [23]:
y_test_df['unique_id'].nunique()

4640

In [25]:
preds_test_df['unique_id'].nunique()

4641

In [20]:
y_hat_fforma = fforma.predict(X_test_df, preds_test_df, y_test_df)

ValueError: Length of values does not match length of index

In [ ]:
# Plot
def plot_prediction(y, y_hat,ax):
    n_y = len(y)
    n_yhat = len(y_hat)
    ds_y = np.array(range(n_y))
    ds_yhat = np.array(range(n_y-n_yhat, n_y))

    ax.plot(ds_y, y, label = 'y')
    ax.plot(ds_yhat, y_hat, label='y_hat')
    
def plot_grid(x,n_row,n_col):
    n_graph = len(x)
    fig, axs = plt.subplots(n_row, n_col, figsize=(5*n_col, 3*n_row))
    plt.xticks(rotation=45)
    
    for i in range(n_graph):
        row = int(np.floor(i/n_col))
        col = i % n_col
        plot_prediction(y=x[i][0],y_hat=x[i][1], ax=axs[row, col])
        flag=1
    plt.show()

In [ ]:
(y_hat['y']-y_hat['y_hat']).mean() #-151 -0.86 -0.57 -102 -81

In [ ]:
ids = y_hat['unique_id'].unique()
plots = []
for i in range(40):
    plot_id = ids[i]
    plot_df = y_hat[y_hat['unique_id']==plot_id]
    y_train = y_train_df[y_train_df['unique_id']==plot_id]
    y = np.concatenate([y_train['y'],plot_df['y']])
    plots.append((y, plot_df['y_hat']))

plot_grid(plots,10,4)